In [1]:
## install needed modules
! pip install geopandas
! pip install fulcrum
! pip install matplotlib
! pip install fiona

In [1]:
## mashup script for batch changes
import math, json
from collections import Counter
from datetime import datetime, timedelta
from geopandas.tools import sjoin
# from pyproj import proj
import pandas as pd
import csv
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import fiona
from fulcrum import Fulcrum

In [2]:
## variables
apiToken = "689a52b9420360bef6af63ad889bcbf4cb34f26e1d1348b2d3a2ff3b71dfe6828d07395724415aa9"
urlBase = 'https://api.fulcrumapp.com/api/v2/'
fulcrum = Fulcrum(key=apiToken)
osTimestamp = datetime.today() - timedelta(days=4)
recordsPerPage = 000
print(fulcrum)

In [3]:
## data I
appID = "0092aa7f-a2e8-4972-8f26-46eb98b23a12" # ID of Combined Tree Invnetory
formdataSource = fulcrum.forms.find(appID)
# get number of pages
## the following divides the total number of records in chunks and downloads and assembles them. Otherwise the fulcrum API allows only downloads of 20,000 records

## count records in app
recordCountSource = fulcrum.forms.find(appID)['form']['record_count']
pagesSource = math.ceil(recordCountSource / recordsPerPage)

## get data I
if pagesSource > 1:
    for p in range(1, pagesSource + 1):
        dataPage = fulcrum.records.search(url_params={'form_id': appID, 'page': p, 'per_page': recordsPerPage})[
            'records']
        if p > 1:
            dataSource.extend(dataPage)
        else:
            dataSource = dataPage
else:
    dataSource = fulcrum.records.search(url_params={'form_id': appID, 'page': 1, 'per_page': recordsPerPage})['records']

In [4]:
## What do we have? Attributes:
type(dataSource)
cticolumns = list(dataSource[0].keys())
cticolumns

['status',
 'version',
 'id',
 'created_at',
 'updated_at',
 'client_created_at',
 'client_updated_at',
 'created_by',
 'created_by_id',
 'updated_by',
 'updated_by_id',
 'created_location',
 'updated_location',
 'created_duration',
 'updated_duration',
 'edited_duration',
 'form_id',
 'project_id',
 'record_series_id',
 'assigned_to',
 'assigned_to_id',
 'form_values',
 'latitude',
 'longitude',
 'altitude',
 'geometry',
 'speed',
 'course',
 'horizontal_accuracy',
 'vertical_accuracy',
 'system_status']

In [9]:
print(dataSource[0]["status"])
status_values = {record["status"] for record in dataSource if "status" in record}
print(status_values)

No action required - لا حاجة لأي إجراء
{'Escalation issue - قضية تصعيد', 'Request of inspection (RFI) - طلب معاينة', 'Rejection of RFI - رفض معاينة', 'Final approval - موافقة نهائية', 'Request of activation - طلب تفعيل', 'Work order - أمر عمل', 'Issue in place - قضية موجودة', 'On hold - قضية عالقة', 'Approval of RFI - موافقة على المعاينة', 'No action required - لا حاجة لأي إجراء', 'Reports', 'Final on hold - نهائي عالق'}


In [25]:
## data II
dqID = "0046e79b-ef91-4b37-99b9-d6340e28094c" # DQ ID
dqformdataSource = fulcrum.forms.find(dqID)
# get number of pages
## the following divides the total number of records in chunks and downloads and assembles them. Otherwise the fulcrum API allows only downloads of 20,000 records

## count records in app
dqrecordCountSource = fulcrum.forms.find(dqID)['form']['record_count']
dqpagesSource = math.ceil(dqrecordCountSource / recordsPerPage)

## get data II
if dqpagesSource > 1:
    for p in range(1, dqpagesSource + 1):
        dataPage = fulcrum.records.search(url_params={'form_id': dqID, 'page': p, 'per_page': recordsPerPage})[
            'records']
        if p > 1:
            dqdataSource.extend(dataPage)
        else:
            dqdataSource = dataPage
else:
    dqdataSource = fulcrum.records.search(url_params={'form_id': dqID, 'page': 1, 'per_page': recordsPerPage})['records']

In [28]:
dqdataSource[0]["status"]
dqstatus_values = {record["status"] for record in dqdataSource if "status" in record}
print(dqstatus_values)

{'No action required - لا حاجة لأي إجراء'}


In [27]:
for record in dqdataSource:
    record["status"] = 'No action required - لا حاجة لأي إجراء' 

In [29]:
# Define the correct project ID
PROJECT_ID = '0092aa7f-a2e8-4972-8f26-46eb98b23a12' # 0092aa7f-a2e8-4972-8f26-46eb98b23a12

for record in dqdataSource:
    record_id = record.get("id")
    
    # Remove unnecessary fields 
    del(record["id"])
    del(record["version"])

    # Ensure the record has the correct project ID
    record["form_id"] = PROJECT_ID  

    # Upload record to Fulcrum
    CREATED = fulcrum.records.create(record)
    # Check CREATED
    #if CREATED and CREATED.get("form_id") == PROJECT_ID:
    #    print(f"✅ Record {record_id} successfully created under Project ID: {PROJECT_ID}")
    #else:
    #    print(f"❌ Failed to create Record {record_id} under Project ID: {PROJECT_ID}")


❌ Failed to create Record 4eb7ba28-9b08-408c-97d0-f5ea7005431c under Project ID: 0092aa7f-a2e8-4972-8f26-46eb98b23a12
❌ Failed to create Record 1e5fa24f-a9e0-414d-9876-c1eb384a40fc under Project ID: 0092aa7f-a2e8-4972-8f26-46eb98b23a12
❌ Failed to create Record 6cc5bd8a-39a5-429b-8fb7-cffae6892999 under Project ID: 0092aa7f-a2e8-4972-8f26-46eb98b23a12
❌ Failed to create Record fce708d8-d18e-4951-97e2-98c37a7ef584 under Project ID: 0092aa7f-a2e8-4972-8f26-46eb98b23a12
❌ Failed to create Record 90f9f3c6-e7c8-4e2a-8522-3d0d3d17b9a2 under Project ID: 0092aa7f-a2e8-4972-8f26-46eb98b23a12
❌ Failed to create Record e541b9c4-e66a-4a1f-85b3-d2c5e29d3c66 under Project ID: 0092aa7f-a2e8-4972-8f26-46eb98b23a12
❌ Failed to create Record c05eea97-b2a1-4c83-8563-fc6349f8bf17 under Project ID: 0092aa7f-a2e8-4972-8f26-46eb98b23a12
❌ Failed to create Record fdcb2c8e-30ea-4f6b-98ec-8f65f6799061 under Project ID: 0092aa7f-a2e8-4972-8f26-46eb98b23a12
❌ Failed to create Record 50ad46cc-7877-4410-be79-bf2458

KeyboardInterrupt: 

In [20]:
CREATED

{'record': {'status': 'No action required - لا حاجة لأي إجراء',
  'version': 1,
  'id': '5868c71c-67b2-41a1-a208-82d0dd25bbe6',
  'created_at': '2025-03-04T11:40:11Z',
  'updated_at': '2025-03-04T11:40:11Z',
  'client_created_at': '2018-02-17T03:44:54Z',
  'client_updated_at': '2023-02-19T08:30:19Z',
  'created_by': '[BP] admin',
  'created_by_id': 'd540ddf0-33db-4fa6-954c-edef0bb8ce30',
  'updated_by': '[BP] admin',
  'updated_by_id': 'd540ddf0-33db-4fa6-954c-edef0bb8ce30',
  'created_location': None,
  'updated_location': None,
  'created_duration': 29,
  'updated_duration': 2,
  'edited_duration': 2,
  'form_id': '0092aa7f-a2e8-4972-8f26-46eb98b23a12',
  'project_id': None,
  'record_series_id': None,
  'changeset_id': None,
  'assigned_to': None,
  'assigned_to_id': None,
  'form_values': {'b96d': '3',
   'd7e8': '1e5fa24f',
   '3fee': {'other_values': [],
    'choice_values': ['Phoenix dactylifera - نخيل البلح']},
   '009b': '12',
   '3a49': 'no',
   '0a3e': {'other_values': [], '

In [29]:
dqcolumns = list(dqdataSource[0].keys())
dqcolumns == cticolumns

True

In [10]:
print(dqdataSource[0]["id"])
print(dqdataSource[1]["id"])
print(dqdataSource[0]["project_id"])
print(dqdataSource[1]["project_id"])

4eb7ba28-9b08-408c-97d0-f5ea7005431c
1e5fa24f-a9e0-414d-9876-c1eb384a40fc
None
None


In [5]:
## Duplicated trees?
ICTres = list(set(record["id"] for record in dataSource))
DQres = list(set(record["id"] for record in dqdataSource))
common_trees = list(set(ICTres) & set(DQres)) or [0]
common_trees

NameError: name 'dataSource' is not defined

In [5]:
## Upload DQ data to CTI data (Version I ~ 01)

import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Define the correct project ID
PROJECT_ID = '0092aa7f-a2e8-4972-8f26-46eb98b23a12'

for record in dqdataSource:
    record_id = record.get("id", "Unknown")  # Store before deleting

    # Remove unnecessary fields (such as 'version' and 'id')
    record.pop("version")
    record.pop("id")

    # Log the record before modification for debugging
    logging.info(f"Record {record_id} before modification: {record.get('form_id', 'Not set')}")

    # Ensure the record has the correct project ID
    record["form_id"] = PROJECT_ID  

    # Log the record after assigning the project ID for debugging
    logging.info(f"Record {record_id} after setting project_id: {record['form_id']}")

    # Upload record to Fulcrum
    CREATED = fulcrum.records.create(record)

    # Log the response for further debugging
    if CREATED and "form_id" in CREATED:
        logging.info(f"✅ Record {record_id} successfully created under Project ID: {PROJECT_ID}")
    else:
        logging.error(f"❌ Failed to create Record {record_id} under Project ID: {PROJECT_ID}")


2025-03-04 11:00:02,358 - INFO - Record 4eb7ba28-9b08-408c-97d0-f5ea7005431c before modification: 0046e79b-ef91-4b37-99b9-d6340e28094c
2025-03-04 11:00:02,359 - INFO - Record 4eb7ba28-9b08-408c-97d0-f5ea7005431c after setting project_id: 0092aa7f-a2e8-4972-8f26-46eb98b23a12
2025-03-04 11:00:03,335 - ERROR - ❌ Failed to create Record 4eb7ba28-9b08-408c-97d0-f5ea7005431c under Project ID: 0092aa7f-a2e8-4972-8f26-46eb98b23a12
2025-03-04 11:00:03,338 - INFO - Record 1e5fa24f-a9e0-414d-9876-c1eb384a40fc before modification: 0046e79b-ef91-4b37-99b9-d6340e28094c
2025-03-04 11:00:03,339 - INFO - Record 1e5fa24f-a9e0-414d-9876-c1eb384a40fc after setting project_id: 0092aa7f-a2e8-4972-8f26-46eb98b23a12
2025-03-04 11:00:04,120 - ERROR - ❌ Failed to create Record 1e5fa24f-a9e0-414d-9876-c1eb384a40fc under Project ID: 0092aa7f-a2e8-4972-8f26-46eb98b23a12
2025-03-04 11:00:04,122 - INFO - Record 6cc5bd8a-39a5-429b-8fb7-cffae6892999 before modification: 0046e79b-ef91-4b37-99b9-d6340e28094c
2025-03-04 

KeyboardInterrupt: 

In [10]:
print(record["form_id"])

0092aa7f-a2e8-4972-8f26-46eb98b23a12


In [49]:
## Merge DQ data to CTI data (Version II) 
merged_dataSource = dataSource + dqdataSource
print(len(merged_dataSource))
print(len(dataSource)+len(dqdataSource))
mcolumns = list(merged_dataSource[0].keys())
print(mcolumns == dqcolumns)

302787
302787
True


In [ ]:
# Send the updated dataSource (which now includes dqdataSource) to the server (Version II ~ 02)
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

for record in merged_dataSource:
    try:
        # Send the record to the server
        response = fulcrum.records.create(record)
        logging.info(f"Record {record['id']} sent to server successfully.")
    except Exception as e:
        logging.error(f"Error sending record {record['id']} to server: {e}")